In [1]:
import matplotlib.pyplot as plt
from tensorflow import keras
from keras import backend as K
from keras.applications import InceptionV3

2023-11-16 00:36:40.140256: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-16 00:36:40.165100: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-16 00:36:40.414579: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-16 00:36:40.414625: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-16 00:36:40.415937: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
import numpy as np
# from keras.applications import VGG16

# PREPROCESSING

In [3]:
img_generator = keras.preprocessing.image.ImageDataGenerator(
    brightness_range=(0.5, 1),
    channel_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1.0 / 255,
    validation_split=0.3
)

In [7]:
root_dir = "../101_ObjectCategories/"
img_generator_flow_train = img_generator.flow_from_directory(
    directory=root_dir,
    target_size=(224, 224),
    batch_size=32,
    shuffle=True,
    subset="training"
)

Found 6444 images belonging to 102 classes.


In [8]:
img_generator_flow_valid = img_generator.flow_from_directory(
    directory=root_dir,
    target_size=(224, 224),
    batch_size=32,
    shuffle=True,
    subset="validation"
)

Found 2700 images belonging to 102 classes.


# LOAD PRETRAINED MODEL

In [9]:
base_model = InceptionV3(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# freeze (weights) in model's lower convolutional layers

In [10]:
base_model.trainable = False

# add custom classifier with several layers of trainable parameters to model

In [14]:
model = keras.models.Sequential([
    base_model,
    keras.layers.MaxPooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(102, activation='softmax')
])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 2, 2, 2048)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 102)               835686    
                                                                 
Total params: 22638470 (86.36 MB)
Trainable params: 835686 (3.19 MB)
Non-trainable params: 21802784 (83.17 MB)
_________________________________________________________________


# train classifier layers on training data available for task

In [16]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(
    img_generator_flow_train,
    validation_data=img_generator_flow_valid,
    epochs=3
)

Epoch 1/3
108/202 [===============>..............] - ETA: 1:43 - loss: 3.8290 - categorical_accuracy: 0.5680

# fine-tune hyper parameters and unfreeze more layers as needed

In [ ]:
base_model.trainable = True

model.compile